In [4]:
#Step 1. 필요한 모듈과 라이브러리를 로딩합니다.
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import numpy as np
import pandas as pd   
import os

#아래 모듈들이 중요합니다.
import cv2          # pip install opencv_python , pip install opencv-contrib-python
import pyautogui    # pip install pyautogui
import imutils      # pip install imutils

#Step 2.사용자에게 안내를 보여주고 조회할 날짜를 입력받습니다.
print("=" *80)
print("이 프로그램은 국세청 홈텍스 사이트에서 매출 전자세금계산서(전자계산서)를")
print("조회하여 엑셀 파일로 저장하는 프로그램입니다")
print("이 프로그램은 서진수가 교육용으로 제작하였으며 이 프로그램에 대한 문의는")
print("seojinsu@gmail.com 으로 보내주시기 바랍니다~^^")
print("=" *80)

print("=" *28 , '기간 조회시 참고하세요' , "="*28)
print('홈텍스에서의 전자세금계산서(전자계산서) 조회기간은 3개월입니다')
print("단 3개월 이상 자료는 '월/분기별 목록조회'화면에서 조회할 수 있습니다") 
print('공급(받는)자 등록번호 입력시 1년이내 조회 가능)')
print("=" *80)

print("\n")
start_date = input('1.조회 시작 날짜를 입력하세요(예:2019-12-01): ')
end_date = input('2.조회 종료 날짜를 입력하세요(예:2019-12-31): ')
print("\n")

print('요청하신 내용으로 홈텍스에 로그인 한 후 데이터 수집을 시작합니다')
print('잠시만 기다려 주세요~~^^')

#Step 3. 크롬 드라이버 설정하고 홈택스에 접속합니다.
query_url='https://www.hometax.go.kr//'
chrome_path = "c:/temp/chromedriver_85/chromedriver.exe"
driver = webdriver.Chrome(chrome_path)
driver.get(query_url)
driver.maximize_window()
time.sleep(5)

# 상단의 로그인 버튼 클릭하기
driver.find_element_by_id('textbox81212912').click()
time.sleep(2)

#Step 4.공인 인증서로 로그인합니다.
#같은 그림 좌표 찾는 함수
def match_center_loc(screen, template):
    res = cv2.matchTemplate(screen, template, cv2.TM_CCOEFF_NORMED)
    min_val,max_val,min_loc, max_loc = cv2.minMaxLoc(res)
    top_left = max_loc
    h,w = template.shape[:2]
    x, y = int(top_left[0] + w/2), int(top_left[1] + h/2)
    return x,y

#전체 화면 캡쳐하기
time.sleep(5)
image1 = pyautogui.screenshot()
image2 = cv2.cvtColor(np.array(image1), cv2.COLOR_RGB2BGR)
cv2.imwrite("c:\\data\\image\\full_screen_1.png", image2)
time.sleep(1)

screenshot_1 = cv2.imread('c:\\data\\image\\full_screen_1.png', cv2.IMREAD_COLOR)
img_0 = cv2.imread('c:\\data\\image\\login.jpg', cv2.IMREAD_COLOR)     # 로그인버튼
img_1 = cv2.imread('c:\\data\\image\\name.jpg', cv2.IMREAD_COLOR)      # 회사이름
img_2 = cv2.imread('c:\\data\\image\\password.jpg', cv2.IMREAD_COLOR)  # 비밀번호 입력창 
img_3 = cv2.imread('c:\\data\\image\\enter.jpg', cv2.IMREAD_COLOR)     # 확인버튼 

# 공인인증서로 로그인 버튼 클릭하기 
driver.switch_to.frame('txppIframe')
driver.find_element_by_id('trigger38').click()
time.sleep(5)

# 공인인증서 화면에서 회사이름 선택하고 클릭
image1 = pyautogui.screenshot()
image2 = cv2.cvtColor(np.array(image1), cv2.COLOR_RGB2BGR)
cv2.imwrite("c:\\data\\image\\full_screen_2.png", image2)
time.sleep(1)
screenshot_2 = cv2.imread('c:\\data\\image\\full_screen_2.png', cv2.IMREAD_COLOR)

match_pos_1 = match_center_loc(screenshot_2, img_1)
pyautogui.moveTo(match_pos_1)
pyautogui.click()
time.sleep(1)

# 암호인증 창에 암호 입력하기
match_pos_2 = match_center_loc(screenshot_2, img_2)
pyautogui.moveTo(match_pos_2)
pyautogui.click()
password = '이곳에 여러분의 공인인증서 암호를 입력하세요'
pyautogui.typewrite(password)

# 암호 입력 후 확인 버튼 클릭하기
match_pos_3 = match_center_loc(screenshot_2, img_3)  # 인증서암호 넣고 확인 버튼 누르기
pyautogui.moveTo(match_pos_3)
pyautogui.click()
time.sleep(5)

#Step 5. 전자세금계산서 조회를 시작합니다.
#조회/발급 메뉴 클릭하기
driver.find_element_by_link_text('조회/발급').click()
time.sleep(2)

# 목록조회 -> 발급목록 조회 클릭하기
driver.switch_to.frame('txppIframe')

driver.find_element_by_xpath('//*[@id="a_0104020000"]').click()
driver.find_element_by_xpath('//*[@id="a_0104020100"]').click()
time.sleep(5)

#전자 계산서 클릭하기
radio = driver.find_element_by_id("radio6_input_0")
radio.click()
time.sleep(1)

# 기간 입력하여 조회하기
s_input_area = driver.find_element_by_id('inqrDtStrt_input')
s_input_area.click()
s_input_area.clear()
for a in start_date :
    s_input_area.send_keys(a)
    time.sleep(0.2)

time.sleep(1)

e_input_area = driver.find_element_by_id('inqrDtEnd_input')
e_input_area.click()
time.sleep(1)
e_input_area.clear()
for a in end_date :
    e_input_area.send_keys(a)
    time.sleep(0.2)

time.sleep(1)

driver.find_element_by_id('trigger50').click()

time.sleep(3)

#Step 6. 조회된 결과를 엑셀 형태의 파일로 저장합니다.
#내려받기 버튼 클릭하기
driver.find_element_by_id('trigger55').click()
time.sleep(2)

driver.switch_to.frame('UTEETBDA17_iframe')
driver.find_element_by_id('btnProcess').click()
time.sleep(2)

#엑셀 버튼 클릭하기
driver.find_element_by_id('trigger4').click()
time.sleep(2)

#참고 : alert창의 내용 확인하기
#alert = driver.switch_to.alert
#message = alert.text
#print("Alert 창 내용: "+ message )
#time.sleep(2)

#Alert 창의 확인 버튼 클릭하기
alert = driver.switch_to.alert
alert.accept()
time.sleep(2)

#참고 : alert 창을 취소하고 싶을 경우
#alert = driver.switch_to.alert
#alert.dismiss()
#time.sleep(2)

#취소 버튼 클릭한 후 창 닫기
driver.find_element_by_id('trigger10004').click()
time.sleep(2)

driver.close()

이 프로그램은 국세청 홈텍스 사이트에서 매출 전자세금계산서(전자계산서)를
조회하여 엑셀 파일로 저장하는 프로그램입니다
이 프로그램은 서진수가 교육용으로 제작하였으며 이 프로그램에 대한 문의는
seojinsu@gmail.com 으로 보내주시기 바랍니다~^^
============================ 기간 조회시 참고하세요 ============================
홈텍스에서의 전자세금계산서(전자계산서) 조회기간은 3개월입니다
단 3개월 이상 자료는 '월/분기별 목록조회'화면에서 조회할 수 있습니다
공급(받는)자 등록번호 입력시 1년이내 조회 가능)


1.조회 시작 날짜를 입력하세요(예:2019-12-01): 2019-12-01
2.조회 종료 날짜를 입력하세요(예:2019-12-31): 2019-12-31


요청하신 내용으로 홈텍스에 로그인 한 후 데이터 수집을 시작합니다
잠시만 기다려 주세요~~^^
